In [37]:
from vespa.package import ApplicationPackage, Document, Field, OnnxModel, Schema

app_package = ApplicationPackage(
    name="onnxexample",
    schema=[
        Schema(
            name="onnxexample",
            document=Document(
                fields=[
                    Field(
                        name="document_id",
                        type="string",
                        indexing=["attribute", "summary"],
                    ),
                    Field(name="text", type="string", indexing=["index", "summary"]),
                ]
            ),
            onnx_model=OnnxModel(
                model_name="onnx_model", model_file_path="/app/transformer.onnx"
            ),
        )
    ],
    stateless_model_evaluation=True,
)

In [38]:
from vespa.deployment import VespaDocker

vespa_docker = VespaDocker(
    volumes=["/Users/thomas/repos/pyvespa/dev/transformer.onnx:/app/transformer.onnx"]
)
app = vespa_docker.deploy(application_package=app_package)

Waiting for configuration server, 0/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 0/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 5/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 10/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Application is up!
Finished deployment.


In [45]:
app.get_model_endpoint(model_id="onnxexample")

{'status_code': 404, 'message': 'Not Found'}

In [40]:
!curl http://localhost:8080/models-evaluation/

<html>
<head>
<meta http-equiv="Content-Type" content="text/html;charset=ISO-8859-1"/>
<title>Error 404</title>
</head>
<body>
<h2>HTTP ERROR: 404</h2>
<p>Problem accessing /models-evaluation/. Reason:
<pre>    No binding for URI &apos;http://localhost:8080/models-evaluation/&apos;.</pre></p>
<hr/>
</body>
</html>


In [41]:
!vespa curl /models-evaluation/v1/

<html>
<head>
<meta http-equiv="Content-Type" content="text/html;charset=ISO-8859-1"/>
<title>Error 404</title>
</head>
<body>
<h2>HTTP ERROR: 404</h2>
<p>Problem accessing /models-evaluation/v1/. Reason:
<pre>    No binding for URI &apos;http://127.0.0.1:8080/models-evaluation/v1/&apos;.</pre></p>
<hr/>
</body>
</html>


In [46]:
vespa_docker.container.stop()
vespa_docker.container.remove()